In [144]:
import geopandas as gpd
import pandas as pd
import warnings
from shapely.geometry import Point

# Apaga todas las advertencias (warnings)
warnings.filterwarnings("ignore")

In [145]:
# nodes = gpd.read_file('/app/data/red_ajuste_final_final_ahorasiquesi/nodes')
# edges = gpd.read_file('/app/data/red_ajuste_final_final_ahorasiquesi2/edges')
# quadp = gpd.read_file('/app/data/red_ajuste_final_final_ahorasiquesi2/cuadrante_prueba')

# nodes.to_parquet('/app/data/red_ajuste_final_final_ahorasiquesi/nodes.parquet')
# edges.to_parquet('/app/data/red_ajuste_final_final_ahorasiquesi2/edges.parquet')
# quadp.to_parquet('/app/data/red_ajuste_final_final_ahorasiquesi2/cuadrante_prueba.parquet')

# nodes = gpd.read_parquet('/app/data/red_ajuste_final_final_ahorasiquesi/nodes.parquet')
nodes = gpd.read_parquet('/app/assets/streets/future/nodes.parquet')
edges = gpd.read_parquet('/app/data/red_ajuste_final_final_ahorasiquesi2/edges.parquet')
quadp = gpd.read_parquet('/app/data/red_ajuste_final_final_ahorasiquesi/cuadrante_prueba.parquet')

In [146]:
# nodes_cols = ['osmid', 'geometry']
# nodes = nodes[nodes_cols]
# mask = (nodes['osmid']==0)
# area_nodes = nodes[mask]
# area_nodes['osmid'] = ('2024'+ area_nodes.index.astype(str)).astype(int)
# nodes = pd.concat([nodes[~mask], area_nodes])

In [147]:
edges_cols = ['u', 'v', 'key', 'osmid', 'geometry']
edges = edges[edges_cols]

In [148]:
area_edges = gpd.sjoin(edges, quadp, how='inner', op='intersects')

In [149]:
area_edges[['u', 'v']] = pd.NA

In [150]:
area_edges.shape

(4512, 7)

In [151]:
# Añade una nueva columna llamada "begin" que contiene el primer punto de cada línea
area_edges['begin'] = area_edges['geometry'].apply(lambda geom: Point(geom.coords[0]))

# Añade una nueva columna llamada "end" que contiene el último punto de cada línea
area_edges['end'] = area_edges['geometry'].apply(lambda geom: Point(geom.coords[-1]))

# Función para encontrar el punto más cercano en el GeoDataFrame "nodes"
def find_nearest_node(point):
    nearest = nodes['geometry'].apply(lambda x: point.distance(x)).idxmin()
    return nodes.loc[nearest, 'osmid']

# Aplicamos la función para encontrar el "source" y "destination" para cada línea en "new_edges"
area_edges['u'] = area_edges['begin'].apply(find_nearest_node)
area_edges['v'] = area_edges['end'].apply(find_nearest_node)

In [152]:
outside_edges = edges.loc[~edges.index.isin(area_edges.index), edges_cols]

In [153]:
out_edges = pd.concat([outside_edges, area_edges])[edges_cols]

In [154]:
out_edges['length'] = out_edges.length

In [155]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

# Función para invertir la geometría y mantener el formato LINESTRING
def reverse_geometry(geometry):
    return LineString(geometry.coords[::-1])

nodes_df, edges_df = nodes.copy(), out_edges.copy()

print(edges_df.shape)

# Asegúrate de que la columna 'u' siempre sea menor que 'v'
edges_df['sorted_edges'] = edges_df.apply(lambda row: tuple(sorted([row['u'], row['v']])), axis=1)

edges_df.drop_duplicates(subset=['sorted_edges'], inplace=True, keep='first')
print(edges_df.shape)

# # ['u', 'v', 'key', 'from', 'to', 'osmid', 'length', 'geometry','sorted_edges']

reverse_edges = edges_df[['key','osmid', 'geometry']]
reverse_edges['geometry'] = reverse_edges['geometry'].apply(reverse_geometry)
reverse_edges[['u', 'v']]  = edges_df[['v', 'u']]
print(reverse_edges.columns)

# cols = ['u', 'v', 'key', 'osmid', 'from', 'to', 'geometry']
edges_df = pd.concat([edges_df[reverse_edges.columns], reverse_edges])
# edges_df.shape

edges_df.head(3)
print(edges_df.shape)

(81405, 6)
(40721, 7)
Index(['key', 'osmid', 'geometry', 'u', 'v'], dtype='object')
(81442, 5)


In [156]:
edges_df['length'] = edges_df.length

In [157]:
edges_df.reset_index(inplace=True)

In [158]:
def format_net_nodes(nodes_df):
    nodes = pd.DataFrame(
        {
            'osmid': nodes_df['osmid'].astype(int),
            'lat' : nodes_df.geometry.y.astype(float),
            'lon' : nodes_df.geometry.x.astype(float),
            'y' : nodes_df.geometry.y.astype(float),
            'x' : nodes_df.geometry.x.astype(float),
        }
    )
    nodes['id'] = nodes['osmid'].values

    nodes = gpd.GeoDataFrame(data=nodes, geometry=nodes_df.geometry)
    nodes.set_index('osmid', inplace=True)
    return nodes

net_nodes = format_net_nodes(nodes)

def format_net_edges(edges_df):
    edges = pd.DataFrame(
        {
            'u': edges_df['u'].astype(int),
            'v': edges_df['v'].astype(int),
            'from': edges_df['u'].astype(int),
            'to': edges_df['v'].astype(int),
            'osmid': edges_df['osmid'].astype(int),
            'length': edges_df['length'].astype(float)
        }
    )
    edges['key'] = 0
    edges['key'] = edges['key'].astype(int)
    edges = gpd.GeoDataFrame(data=edges, geometry=edges_df.geometry)
    edges.set_index(['u', 'v', 'key'], inplace=True)
    return edges

net_edges = format_net_edges(edges_df)

In [159]:
import pandana as pdna

nodes_df = net_nodes
edges_df = net_edges

net = pdna.Network(
    nodes_df['lon'],
    nodes_df['lat'],
    edges_df['from'],
    edges_df['to'],
    edges_df[['length']]
)

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 36894
Setting CH edge vector of size 81442
Range graph removed 81442 edges of 162884
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [162]:
edges_df

from          to      osmid     length  \
u          v          key                                                 
267252253  267252256  0     267252253   267252256  368010828   4.812181   
           3718767746 0     267252253  3718767746  368010828   3.901331   
267252256  8355196537 0     267252256  8355196537  367537909   5.850963   
           3718673159 0     267252256  3718673159  368010828   6.686680   
           7248696117 0     267252256  7248696117  776869883  11.688149   
...                               ...         ...        ...        ...   
202436800  202436801  0     202436800   202436801  972999344  89.344644   
202436804  7248560572 0     202436804  7248560572  972999344  54.501553   
202436784  1142890025 0     202436784  1142890025   98795720  22.088060   
202436783  202436784  0     202436783   202436784   98795720  12.446319   
7849111007 202436845  0    7849111007   202436845  988969775   7.522870   

                                                                    geometry  
u          v          key                                                     
267252253  267252256  0    LINESTRING (674769.156 5922560.301, 674767.130...  
           3718767746 0    LINESTRING (674769.156 5922560.301, 674771.258...  
267252256  8355196537 0    LINESTRING (674767.130 5922555.936, 674761.516...  
           3718673159 0    LINESTRING (674767.130 5922555.936, 674766.174...  
           7248696117 0    LINESTRING (674767.130 5922555.936, 674778.274...  
...                                                                      ...  
202436800  202436801  0    LINESTRING (672786.192 5921896.768, 672865.586...  
202436804  7248560572 0    LINESTRING (672920.543 5921966.109, 672968.975...  
202436784  1142890025 0    LINESTRING (672947.037 5921438.387, 672927.445...  
202436783  202436784  0    LINESTRING (672958.077 5921444.134, 672947.037...  
7849111007 202436845  0    LINESTRING (672947.789 5921452.071, 672940.959...  

[81442 rows x 5 columns]

In [ ]:
nodes['x'] = nodes.geometry.x
nodes['y'] = nodes.geometry.y

In [ ]:
# nodes.to_parquet('/app/assets/streets/future/nodes.parquet')

In [163]:
edges_df.reset_index().to_parquet('/app/assets/streets/future/edges.parquet')